# Getting our Schema in TigerGraph

Now that we have a good idea what our schema will look like, it's time to get it set up in TigerGraph. Once that's done, we'll map our data file and finally load our data.

We're going to look at two different ways of defining our schema, in the GraphStudio UI, and via GSQL code.

Before we can get into creating the schema, we need to talk about how schema works in TigerGraph. The main thing to note here is that you can have both a **Global** schema and a **Graph** schema. Because you can have multiple **Graphs** on one TigerGraph solution, those **Graphs** need to be able to maintain a schema specific to them.

## Global Schema

This is an overarching schema that can be used by as many or as few **Graphs** as you desire. The purpose of the **Global Schema** is to give you a common schema that you CAN pull from when setting up **Graphs**. This is helpful if you have common elements between your different **Graphs** and want those common elements to have the same schema. For example, my **Global Schema** might contain the entirety of information about my supply chain. Everything from suppliers, warehouses, transportation lines, user orders, product information, user shipping information. However, one of our **Graphs** might want to only contain information about the suppliers, warehouses, and transportation as that side of the business doesn't need access to customer information. I could use just those elements and their edges from the **Global Schema** to populate this **Graph** without having to include all fo the information about the ordering user.

## Graph Schema

As we talked about above, the **Graph Schema** can contain as much or none of the **Global Schema** as you desire. In addition, the **Graph Schema** can contain elements that are not in the **Global Schema**. From the example above, I want to keep track of the manufacturing side of my supply chain. I can include the suppliers, warehouses, and transportation from the **Global Schema**, but I can also add in something like weather forecasts and supply forecasts that could be used to predict disruptions to my manufacturing chain. This data can exist in the **Graph Schema** for the Manufacturing Graph, but not in the **Global Schema**

## Naming Conventions

Let's talk naming conventions really quickly. When building a complicated graph, you will have a variety of Vertex Types, Edge Types, and attributes. I recommend that you use different naming conventions for each so that way you can easily remember what aspect of the graph a variable is based on how it's named. This will come in very handy when writing queries.

I'm going to share the naming convention I use, but you should use what's comfortable to you, or what fits any existing work you're trying to integrate into.

<hr>

### Vertex Types

Vertex name describes the concept that the vertex represents

**PascalCase** - all word beginnings are capital letters
- FileLocation
- ContinuationType
- PGPUBNumber

<hr>

### Edge Types

Edge name includes verb describing relationship and denotation of target vertex type

**snake_case** - all words are lowercase and separated by an underscore
- has_patent
- at_location
- from_city

<hr>

### Attributes

Attribute name describes attribute field an can usually just be the column name from the source table (reformatted into your choice naming convention)

**camelCase** - first word is lowercase, subsequent words start with a capital letter
- filingDate
- fullName
- description


## GraphStudio Schema Design

The GraphStudio tool is extremely helpful when you are first defining your schema. The UI makes it very easy to see exactly what the graph schema you are creating will look like. It also provides easy access to all of the options and customizations you can make to a vertex or edge when defining it. The UI is FANTASTIC when you are initially setting up your schema and still figuring out how everything will fit together.

Where the UI lacks is it's repeatability. You don't want to have to spend time clicking around to set up your schema again in the event that you need to re-do it. Luckily that's where GSQL comes in, and even more fortunately, we can export all of the hard work that we do in GraphStudio as GSQL, but we'll get to that later.

### Creating a Graph

We are going to be working within the **Graph Schema** here. When we are doing our initial data explorations, it is easier for us to adjust the **Graph Schema** than the **Global Schema**. Changing the **Global Schema** will require any Graphs using the changed part of the **Global Schema** to be dropped. This means you will have to reload the entirety of the data and re-do the mapping. When changing the **Graph Schema**, only the vertices or edges of the changed type will have to be dropped. 

<img src="images/create_graph_1.png" width="500px" /> <img src="images/create_graph_2.png" width="250px" /> <img src="images/create_graph_3.png" width="500px" />

### Creating a Vertex

On the **Design Schema** tab, use the **White Plus icon** to define a vertex type.

<img src="images/create_vertex_1.png" width="500px" />

This will bring up a panel allowing us to further describe the **Vertex Type**. I'll walk through each of the fields with a vertex from our dataset.

<img src="images/create_vertex_2.png" width="500px" />

### The Application Vertex

As a refresher we'll use the **Application** vertex here. Which looks like this:

**Application**
|Attribute|Type|
|---|---|
|applicationNumber (primary_id)|string|
|filingDate|datetime|
|confirmationNumber|string|
|docketNumber|string|
|title|string|

I was able to extract the **Type** information from the Appendix of our data.

<img src="images/application_data_schema.png" width="500px" />

You'll notice that I used **DateTime** even though *filingDate* specified a **float**. In the original database, that *filingDate* is stored as epoch time as a float, but formatted as a date as we can see from the `%td` formatting specifier. Looking through the actual data files will show that the *filingDate* field is formatted as `year-month-day` (1968-11-18) and that doesn't look like any float I've ever seen but it certainly looks like a DateTime. Under the hood, TigerGraph stores DateTime as an **int**, however using **dateTime** allows us to use **dateTime** functions in our queries without first having to convert from **int**.

#### Vertex Type and ID

Of the fields on the *Create Vertex* tab, only two of them are REQUIRED. Those are the **Vertex type name** and **Primary id**. The **Vertex type name** is the name that we'll use to refer to that vertex type by. **Primary id** is where we set the name of the field that will contain the Primary ID of the vertex. The variable type of the Primary ID will also need to be set to the relevant value for the data type being loaded in.

<img src="images/create_vertex_3.png" width="500px" />

Additionally, you will find a checkbox for **As attribute**. Normally, the **Primary id** of a vertex is not accessible when querying the vertex. If we would like to be able to access the **Primary id** like we would an attribute, then we will need to check the **As attribute** box. For example, with our **Application** vertex type here, we'll be using the *applicationNumber* as the **Primary id**. WITHOUT it set as an attribute, we could return **Application** from a query and read all of it's attributes, but we wouldn't be able to see the *applicationNumber*. Setting **As attribute** will allow us to return that *applicationNumber* as well. As an example of when this would not be necessary, recall the **PTAEvent** vertex type from earlier. We had to generate a unique id for that vertex because one did not exist in our source dataset. This ID isn't any important other than providing a unique identifier to the graph engine (if it was we wouldn't have to generate it). In this scenario, we wouldn't need to select **As attribute** because we don't care what that unique ID is, we just care about the attribute information contained in the rest of the vertex.

#### Attributes

<img src="images/create_vertex_4.png" width="500px" />

Attributes contain the actual informational contents of a **Vertex**. These are the values that will be available to you when querying a vertex. 

- **Attribute name** is pretty self explanatory, this is the name that will be used to reference the attribute when querying and loading data.
- **Attribute type** is the variable type of the attribute. You can review the list of available types [here](https://docs.tigergraph.com/gsql-ref/current/querying/data-types).
- **Default value** this is an override for the default value that an attribute will assume if no data is provided. You do not need ot set this unless you are looking to override the default value defined [here](https://docs.tigergraph.com/gsql-ref/current/querying/data-types#_base_types)
- **Index** this checkbox will denote that this attribute is intended to be used as a secondary index for the **Vertex type**. You can only have one attribute be an **Index** per **Vertex type**.

<img src="images/create_vertex_5.png" width="500px" />

#### Style

This last section only applies to the GraphStudio interface. Here you can select the **Color** and optional **Icon** that can represent a vertex type.

<img src="images/create_vertex_6.png" width="500px" />

### Creating an Edge

Select the **Arrow** icon from the toolbar at the top of the **Design Schema** page. The **Arrow** will turn blue indicating that it is active.

<img src="images/create_edge_1.png" width="500px" />

Select the *Source* vertex, then the *Target* vertex. If your edge is un-directed the selection order does not matter.

<img src="images/create_edge_2.png" width="500px" />

#### Edge type name

This is the name that we will be using to reference the **Edge type**.

#### Directed

This is where we can select wether or not an edge is **Directed**. When selecting directed edge, the **Reverse Edge** checkbox and **Reverse edge type** field will appear. 

<img src="images/create_edge_3.png" width="500px" />

#### Source and target vertex types

Here we declare the **Source** and **Target** vertex types. Edges can either have a singular **Source** - **Target** pair, or can exist as **Compound Edges** with multiple **Source** - **Target** pairs. 

<img src="images/create_edge_4.png" width="500px" />

The **Source** and **Target** can both be the same **Vertex Type**.

#### Attributes

The attributes selection for our edges is the same as the attributes section for the vertices.

<img src="images/create_edge_5.png" width="500px" />

#### Publish Schema

<img src="images/create_edge_6.png" width="500px" />

Currently the schema only exists within the GraphStudio interface. Use the **Publish Schema** button to save the schema to the TigerGraph server. You will be asked to confirm the schema changes, then the schema will be published.

## GSQL Schema Design

For times when you don't want to click through the interface in order to create your schema, you can do it via TigerGraph's language **GSQL**. **GSQL** can be executed on your TigerGraph server via the **GSQL** terminal or remotely by one of our many TigerGraph connectors. For this example we'll be using [pyTigerGraph](https://github.com/pyTigerGraph/pyTigerGraph) to interface through this Python notebook. The GSQL will remain the same regardless of which connector method you use.

We're going to walk through the same vertex and edge creation that we went through with **Application** and **is_continuation_type** except this time in **GSQL**.

### Creating a Vertex

This is what the GSQL required to create the **Application** vertex looks like:

`CREATE VERTEX Application(PRIMARY_ID id STRING, filingDate DATETIME, confirmationNumber STRING, docketNumber STRING, title STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"`

To simplify things, this is the pattern your most basic vertex deceleration will follow:

`CREATE VERTEX <VertexType>(PRIMARY_ID id <DataType>, <attributeName1> <DataType1>) WITH PRIMARY_ID_AS_ATTRIBUTE="true"`

Additional attributes are separated by commas and placed after the first.

### Creating an Edge

The GSQL to create an edge is extremely similar to that to create a vertex. Here's the **is_continuation_type** edge:

`CREATE UNDIRECTED EDGE is_continuation_type(FROM Application, TO ContinuationIype)`

Let's look at a **Directed** edge for comparison:

`CREATE DIRECTED EDGE has_child(FROM Application, TO Application, date DATETIME) WITH REVERSE_EDGE="reverse_has_child"`

And lastly, the pattern ( [  ] = **Optional** ):

`CREATE DIRECTED|UNDIRECTED EDGE <edge_name>(FROM <VertexType> TO <VertexType>, <attributeName1> <DataType1>) [WITH REVERSE_EDGE=<reverse_edge_name>]`

#### Compound Edges

This is a slightly more advanced concept, but it's going to be useful in our example. We're actually going to modify our schema slightly to simplify edge names. 

This was the previous schema for the location section of our graph:

<img src="images/location_3.png" width="500" />

With the ability to use the same **Edge Name** for multiple sets of Source - Target connection, we can make our schema much simpler. Edges of the same color share the same **Edge Name**.

<img src="images/location_4.png" width="500" />

It's important to note here that we are only doing this when the edges are expressing the same relationship. For example:

<img src="images/compound_1.png" width="300" />

**Application** is linked to both **Type** and **ContinuationType**. Both of these vertices represent a type so why doesn't it make sense to call both edges **has_type**? It's because these are different concepts. **Type** is not the same as **ContinuationType**. If there was another concept in our graph that related back to our **Type** variable, then we could call that connection **has_type** because it is referencing the same **Target vertex** of the other **has_type** edge.

Let's take a look at what a **Compound Edge** looks like.

`CREATE UNDIRECTED EDGE in_code (FROM Address, TO PostalCode | FROM Correspondence, TO PostalCode | FROM City TO, PostalCode)`

Notice that pipes separate the individual **Source** - **Target** pairs.

## The Full Schema

Now it's time to declare our whole schema. I'm going to separate the declarations by vertices and edges, but there is nothing stopping you from declaring your whole schema in one statement.

Read through this and see how it lines up with the visual schema that we created earlier.

### Vertices

```
CREATE VERTEX Application(PRIMARY_ID id STRING, filingDate DATETIME, confirmationNumber STRING, docketNumber STRING, title STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX ContinuationType(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX USPCClass(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX USPCSubclass(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX EventCode(PRIMARY_ID id STRING, description STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX PTAEvent(PRIMARY_ID id STRING, description STRING, applicantDelay FLOAT, usptoDelay FLOAT) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX ExtensionIndicator(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX PTASummary(PRIMARY_ID id STRING, ptoDelayA FLOAT, ptoDelayB FLOAT, ptoDelayC FLOAT, overlapDelay FLOAT, nonOverlapDelay FLOAT, manualAdjustment FLOAT, applicationDelay FLOAT, PTA FLOAT) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX PTESummay(PRIMARY_ID id STRING, ptoAdjustment FLOAT, ptoDelay FLOAT, applicantDelay FLOAT, PTE FLOAT) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX ApplicationStatus(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX PatentNumber(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Examiner(PRIMARY_ID id STRING, fullName STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX ArtUnit(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Attorney(PRIMARY_ID id STRING, firstName STRING, middleName STRING, lastName STRING, suffix STRING, phone STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX PracticeCategory(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX SmallEntity(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX First_toFile(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX FileLocation(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX PGPUBNumber(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX WIPONumber(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Inventor(PRIMARY_ID id STRING, first STRING, middle STRING, last STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX ForeignParent(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX City(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Region(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Country(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX PostalCode(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Correspondence(PRIMARY_ID id STRING, name STRING, customerNumber STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Address(PRIMARY_ID id STRING, line1 STRING, line2 STRING, line3 STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
```

### Edges

```
CREATE DIRECTED EDGE has_child(FROM Application, TO Application, date DATETIME) WITH REVERSE_EDGE="reverse_has_child"
CREATE DIRECTED EDGE has_parent(FROM Application, TO Application, date DATETIME) WITH REVERSE_EDGE="reverse_has_parent"
CREATE UNDIRECTED EDGE is_continuation_type(FROM Application, TO ContinuationType)
CREATE UNDIRECTED EDGE has_class(FROM Application, TO USPCClass)
CREATE UNDIRECTED EDGE has_subclass(FROM Application, TO USPCSubclass)
CREATE DIRECTED EDGE is_subclass(FROM USPCSubclass, TO USPCClass) WITH REVERSE_EDGE="reverse_is_subclass"
CREATE UNDIRECTED EDGE has_code(FROM Application, TO EventCode, date DATETIME)
CREATE UNDIRECTED EDGE has_pta_event(FROM Application, TO PTAEvent, date DATETIME)
CREATE UNDIRECTED EDGE is_extension(FROM PTAEvent, TO ExtensionIndicator)
CREATE DIRECTED EDGE has_start(FROM PTAEvent, TO PTAEvent) WITH REVERSE_EDGE="reverse_has_start"
CREATE UNDIRECTED EDGE has_pta_summary(FROM Application, TO PTASummary)
CREATE UNDIRECTED EDGE has_pte_summary(FROM Application, TO PTESummay)
CREATE UNDIRECTED EDGE has_status(FROM Application, TO ApplicationStatus, date DATETIME)
CREATE UNDIRECTED EDGE has_patent(FROM Application, TO PatentNumber, date DATETIME)
CREATE UNDIRECTED EDGE has_examiner(FROM Application, TO Examiner)
CREATE UNDIRECTED EDGE from_unit(FROM Examiner, TO ArtUnit)
CREATE UNDIRECTED EDGE has_attorney(FROM Application, TO Attorney)
CREATE UNDIRECTED EDGE has_practice_category(FROM Attorney, TO PracticeCategory)
CREATE UNDIRECTED EDGE is_small(FROM Application, TO SmallEntity)
CREATE UNDIRECTED EDGE follows_ftf(FROM Application, TO First_toFile)
CREATE UNDIRECTED EDGE at_location(FROM Application, TO FileLocation, date DATETIME)
CREATE UNDIRECTED EDGE hasIGPUB(FROM Application, TO PGPUBNumber, date DATETIME)
CREATE UNDIRECTED EDGE hasIIPO(FROM Application, TO WIPONumber, date DATETIME)
CREATE UNDIRECTED EDGE filed_application(FROM Application, TO Inventor, rank INT)
CREATE UNDIRECTED EDGE has_foreign_parent(FROM Application, TO ForeignParent, date DATETIME)
CREATE UNDIRECTED EDGE from_city(FROM Inventor, TO City)
CREATE UNDIRECTED EDGE from_region(FROM Inventor, TO Region)
CREATE UNDIRECTED EDGE from_country(FROM Inventor, TO Country)
CREATE UNDIRECTED EDGE filed_country(FROM ForeignParent, TO Country)
CREATE UNDIRECTED EDGE has_correspondence(FROM Inventor, TO Correspondence)
CREATE UNDIRECTED EDGE has_address(FROM Address, TO Correspondence)
CREATE UNDIRECTED EDGE in_code(FROM City, TO PostalCode | FROM Correspondence, TO PostalCode | FROM Address, TO PostalCode)
CREATE UNDIRECTED EDGE in_region(FROM PostalCode, TO Region | FROM Correspondence, TO Region | FROM Address, TO Region)
CREATE UNDIRECTED EDGE in_country(FROM PostalCode, TO Country | FROM Country, TO Region | FROM Correspondence, TO Country | FROM Address, TO Country)
CREATE UNDIRECTED EDGE in_city(FROM Region, TO City | FROM Correspondence, TO City | FROM Address, TO City)
```

## Publishing the Schema with GSQL

We'll use [pyTigerGraph](https://github.com/pyTigerGraph/pyTigerGraph) for this, but you can do this though any GSQL interface on your TigerGraph server.

### Installing requirements

In [2]:
!pip install pyTigerGraph -q

/bin/bash: pip: command not found


### Connecting to the TigerGraph server

I'm not going to go to into detail here as this login process is thoroughly documented in our [pyTigerGraph 101](https://colab.research.google.com/drive/1fJpcv-q0NLfHj3X1k6Lbwddp8gVVcfES) learning.

The long and skinny of it is that we need to first connect to our TigerGraph server, then we need to securely authenticate with the **Graph** itself. Because the **Graph** can contain actual data (**global** space is just schema, no data) a more secure connection is needed via a token. These tokens can also be used to manage access roles for different users, but you can read about that [here](https://docs.tigergraph.com/tigergraph-server/3.3/user-access/roles-and-privileges).

#### Solution Connection

In [6]:
import pyTigerGraph as tg

# connection parameters
# hostName is the TigerGraph solution URL
hostName = "https://patent-free.i.tgcloud.io"
graphName = "Patents"
userName = "tigergraph"
password = "tigergraph"

# establish the connection to the TigerGraph Solution
conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

# print any current schema so we can verify that we are connected
conn.gsql('LS')

'---- Global vertices, edges, and all graphs\nVertex Types:\nEdge Types:\n\nGraphs:\nJobs:\n\n\nJSON API version: v2\nSyntax version: v1\n'

#### Graph Connection

Once connected to the Solution, we can create the required Secret and Token needed to authenticate with our Graph.

In [ ]:
# set the name of the graph that we want to connect to
conn.graphname = graphName

# create a secret
secret = conn.createSecret()
# use the secret to get a token
authToken = conn.getToken(secret)[0]

# connect to graph with token
conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password, graphname=graphName, apiToken=authToken)

# listing vertex count requires graph authentication and will prove that we're securely connected to the Graph
conn.getVertexCount("*")

#### Clearing the Schema (optional)

Since we're going to be loading in the full schema, we should clear any currently loaded schema from the graph as it will throw an error if we try to create a Vertex or Edge with the same name as one that already exists.

In [ ]:
# Get a list of vertices and edges currently in the graph schema
vertices = conn.getVertexTypes()
edges = conn.getEdgeTypes()

# we need a SCHEMA_CHANGE JOB to change the schema, we're going to put that together in the next couple lines
changeJob = '''CREATE SCHEMA_CHANGE JOB clearGraph FOR GRAPH Patents {'''

for vertex in vertices:
    changeJob += ('''DROP VERTEX ''' + vertex + ';')

for edge in edges:
    changeJob += ('''DROP EDGE ''' + edge + ';')

changeJob += '}'

# print the complete change job
print(changeJob)

# add the job to the graph
print(conn.gsql('''
    USE GRAPH patents
    ''' + changeJob))
# execute the change job
print(conn.gsql('''
    USE GRAPH patents
    RUN SCHEMA_CHANGE JOB clearGraph
    '''))
# delete the change job
print(conn.gsql('''
    USE GRAPH patents
    DROP JOB clearGraph
    '''))

#### Clearing the Graph

Via GSQL, it's much easier for us to create our schema in the **Global** manner, then create a **Graph** utilizing elements from that **Global** schema. Adding schema to an existing **Graph** is possible, it just requites a [SchemaIhange Job](https://docs.tigergraph.com/gsql-ref/3.3/ddl-and-loading/modifying-a-graph-schema) rather than schema definition.

Because this process is creating a new **Graph** from our **Global** schema, we need to drop our old **Graph** because we can't have two graphs with the same name.

Remember how we needed a **Secret** and **Token** to connect to a graph specifically? Once we delete our current Graph and create a new one, we will need to create a new **Secret** and **Token** tied to that Graph so we can connect to it.

In [185]:
print(conn.gsql('DROP GRAPH Patents'))

Semantic Check Fails: The job load_job_10K_all_inventors_csv_1639593830396 depends on graph Patents!
Please drop the job first!
The graph Patents could not be dropped!


#### Loading the Schema

It's time to load in our schema. I'm going to do this in separate parts for vertices and edges, but both could be done in the same statement.

In [8]:
# Vertices
conn.gsql('''
CREATE VERTEX Application(PRIMARY_ID id STRING, filingDate DATETIME, confirmationNumber STRING, docketNumber STRING, title STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX ContinuationType(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX USPCClass(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX USPCSubclass(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX EventCode(PRIMARY_ID id STRING, description STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX PTAEvent(PRIMARY_ID id STRING, description STRING, applicantDelay FLOAT, usptoDelay FLOAT) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX ExtensionIndicator(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX PTASummary(PRIMARY_ID id STRING, ptoDelayA FLOAT, ptoDelayB FLOAT, ptoDelayC FLOAT, overlapDelay FLOAT, nonOverlapDelay FLOAT, manualAdjustment FLOAT, applicationDelay FLOAT, PTA FLOAT) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX PTESummay(PRIMARY_ID id STRING, ptoAdjustment FLOAT, ptoDelay FLOAT, applicantDelay FLOAT, PTE FLOAT) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX ApplicationStatus(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX PatentNumber(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Examiner(PRIMARY_ID id STRING, fullName STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX ArtUnit(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Attorney(PRIMARY_ID id STRING, firstName STRING, middleName STRING, lastName STRING, suffix STRING, phone STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX PracticeCategory(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX SmallEntity(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX First_toFile(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX FileLocation(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX PGPUBNumber(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX WIPONumber(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Inventor(PRIMARY_ID id STRING, first STRING, middle STRING, last STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX ForeignParent(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX City(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Region(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Country(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX PostalCode(PRIMARY_ID id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Correspondence(PRIMARY_ID id STRING, name STRING, customerNumber STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Address(PRIMARY_ID id STRING, line1 STRING, line2 STRING, line3 STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
''')

'Successfully created vertex types: [Application].\nSuccessfully created vertex types: [ContinuationType].\nSuccessfully created vertex types: [USPCClass].\nSuccessfully created vertex types: [USPCSubclass].\nSuccessfully created vertex types: [EventCode].\nSuccessfully created vertex types: [PTAEvent].\nSuccessfully created vertex types: [ExtensionIndicator].\nSuccessfully created vertex types: [PTASummary].\nSuccessfully created vertex types: [PTESummay].\nSuccessfully created vertex types: [ApplicationStatus].\nSuccessfully created vertex types: [PatentNumber].\nSuccessfully created vertex types: [Examiner].\nSuccessfully created vertex types: [ArtUnit].\nSuccessfully created vertex types: [Attorney].\nSuccessfully created vertex types: [PracticeCategory].\nSuccessfully created vertex types: [SmallEntity].\nSuccessfully created vertex types: [First_toFile].\nSuccessfully created vertex types: [FileLocation].\nSuccessfully created vertex types: [PGPUBNumber].\nSuccessfully created ve

In [9]:
# Edges
conn.gsql('''
CREATE DIRECTED EDGE has_child(FROM Application, TO Application, date DATETIME) WITH REVERSE_EDGE="reverse_has_child"
CREATE DIRECTED EDGE has_parent(FROM Application, TO Application, date DATETIME) WITH REVERSE_EDGE="reverse_has_parent"
CREATE UNDIRECTED EDGE is_continuation_type(FROM Application, TO ContinuationType)
CREATE UNDIRECTED EDGE has_class(FROM Application, TO USPCClass)
CREATE UNDIRECTED EDGE has_subclass(FROM Application, TO USPCSubclass)
CREATE DIRECTED EDGE is_subclass(FROM USPCSubclass, TO USPCClass) WITH REVERSE_EDGE="reverse_is_subclass"
CREATE UNDIRECTED EDGE has_code(FROM Application, TO EventCode, date DATETIME)
CREATE UNDIRECTED EDGE has_pta_event(FROM Application, TO PTAEvent, date DATETIME)
CREATE UNDIRECTED EDGE is_extension(FROM PTAEvent, TO ExtensionIndicator)
CREATE DIRECTED EDGE has_start(FROM PTAEvent, TO PTAEvent) WITH REVERSE_EDGE="reverse_has_start"
CREATE UNDIRECTED EDGE has_pta_summary(FROM Application, TO PTASummary)
CREATE UNDIRECTED EDGE has_pte_summary(FROM Application, TO PTESummay)
CREATE UNDIRECTED EDGE has_status(FROM Application, TO ApplicationStatus, date DATETIME)
CREATE UNDIRECTED EDGE has_patent(FROM Application, TO PatentNumber, date DATETIME)
CREATE UNDIRECTED EDGE has_examiner(FROM Application, TO Examiner)
CREATE UNDIRECTED EDGE from_unit(FROM Examiner, TO ArtUnit)
CREATE UNDIRECTED EDGE has_attorney(FROM Application, TO Attorney)
CREATE UNDIRECTED EDGE has_practice_category(FROM Attorney, TO PracticeCategory)
CREATE UNDIRECTED EDGE is_small(FROM Application, TO SmallEntity)
CREATE UNDIRECTED EDGE follows_ftf(FROM Application, TO First_toFile)
CREATE UNDIRECTED EDGE at_location(FROM Application, TO FileLocation, date DATETIME)
CREATE UNDIRECTED EDGE has_PGPUB(FROM Application, TO PGPUBNumber, date DATETIME)
CREATE UNDIRECTED EDGE has_WIPO(FROM Application, TO WIPONumber, date DATETIME)
CREATE UNDIRECTED EDGE filed_application(FROM Application, TO Inventor, rank INT)
CREATE UNDIRECTED EDGE has_foreign_parent(FROM Application, TO ForeignParent, date DATETIME)
CREATE UNDIRECTED EDGE from_city(FROM Inventor, TO City)
CREATE UNDIRECTED EDGE from_region(FROM Inventor, TO Region)
CREATE UNDIRECTED EDGE from_country(FROM Inventor, TO Country)
CREATE UNDIRECTED EDGE filed_country(FROM ForeignParent, TO Country)
CREATE UNDIRECTED EDGE has_correspondence(FROM Inventor, TO Correspondence)
CREATE UNDIRECTED EDGE has_address(FROM Address, TO Correspondence)
CREATE UNDIRECTED EDGE in_code(FROM City, TO PostalCode | FROM Correspondence, TO PostalCode | FROM Address, TO PostalCode)
CREATE UNDIRECTED EDGE in_region(FROM PostalCode, TO Region | FROM Correspondence, TO Region | FROM Address, TO Region)
CREATE UNDIRECTED EDGE in_country(FROM PostalCode, TO Country | FROM Country, TO Region | FROM Correspondence, TO Country | FROM Address, TO Country)
CREATE UNDIRECTED EDGE in_city(FROM Region, TO City | FROM Correspondence, TO City | FROM Address, TO City)
''')

'Successfully created edge types: [has_child].\nSuccessfully created reverse edge types: [reverse_has_child].\nSuccessfully created edge types: [has_parent].\nSuccessfully created reverse edge types: [reverse_has_parent].\nSuccessfully created edge types: [is_continuation_type].\nSuccessfully created edge types: [has_class].\nSuccessfully created edge types: [has_subclass].\nSuccessfully created edge types: [is_subclass].\nSuccessfully created reverse edge types: [reverse_is_subclass].\nSuccessfully created edge types: [has_code].\nSuccessfully created edge types: [has_pta_event].\nSuccessfully created edge types: [is_extension].\nSuccessfully created edge types: [has_start].\nSuccessfully created reverse edge types: [reverse_has_start].\nSuccessfully created edge types: [has_pta_summary].\nSuccessfully created edge types: [has_pte_summary].\nSuccessfully created edge types: [has_status].\nSuccessfully created edge types: [has_patent].\nSuccessfully created edge types: [has_examiner].\

#### Creating the Graph

Here we will declare which vertices and edges that we will include in the Graph that we're setting up.

This will take **~2 minutes** to run

In [10]:
conn.gsql(('CREATE GRAPH Patents(Application, ContinuationType, USPCClass, USPCSubclass, EventCode, PTAEvent, '
'ExtensionIndicator, PTASummary, PTESummay, ApplicationStatus, PatentNumber, Examiner, ArtUnit, Attorney, '
'PracticeCategory, SmallEntity, First_toFile, FileLocation, PGPUBNumber, WIPONumber, Inventor, ForeignParent, '
'City, Region, Country, PostalCode, Correspondence, Address, has_child, has_parent, is_continuation_type, has_class, '
'has_subclass, is_subclass, has_code, has_pta_event, is_extension, has_start, has_pta_summary, has_pte_summary, '
'has_status, has_patent, has_examiner, from_unit, has_attorney, has_practice_category, is_small, follows_ftf, at_location, '
'has_PGPUB, has_WIPO, filed_application, has_foreign_parent, from_city, from_region, from_country, in_country, in_city, '
'filed_country, in_code, in_region, has_correspondence, has_address)'))

'Stopping GPE GSE RESTPP\nSuccessfully stopped GPE GSE RESTPP in 41.870 seconds\nStarting GPE GSE RESTPP\nSuccessfully started GPE GSE RESTPP in 0.086 seconds\nThe graph Patents is created.'

Let's head back into **GraphStudio** and check out the Graph we just created. Select the **Patents** graph from the *Switch Graph* dropdown, and navigate to the **Design Schema** tab.

<img src="images/create_graph_4.png" width="100%" />

Hey, that looks familiar.

#### Connecting to the new Graph

Remember that **Token** and **Secret** are stored on the Graph level, so we need to re-authenticate with the new Graph that we created.

In [11]:
# Capital P this time
graphName = "Patents"

# set the name of the graph that we want to connect to
conn.graphname = graphName

# create a secret
secret = conn.createSecret()
# use the secret to get a token
authToken = conn.getToken(secret)[0]

# connect to graph with token
conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password, graphname=graphName, apiToken=authToken)

# Data Mapping

Once our schema is loaded on the TigerGraph server, we can begin mapping our data to that schema. The data mapping tells TigerGraph which columns of our input files match to which IDs and attributes within our graph. Just like with our schema, I'll walk through doing this via the GraphStudio interface and via GSQL code.

## Mapping Data in GraphStudio

Before we can create our mapping in Graph studio, we need to first upload our data files. Since we're still testing out our mapping, we don't want to go upload all 50gb (~4 hours on free tier) of our data just to find out we need to re-format a field in one of the files and re-upload. We'll use the 10K line files that we created previously. Once we have verified that our mapping and connections look correct, then we can proceed from there. In this case, I would probably move up to 100K lines just to reduce the chance of outliers when we move to the full dataset. Being that some of the files in our dataset contain 100M+ lines, 100K is still a relatively small fraction of our dataset.

### Uploading Files

The first step is to get our data files on the TigerGraph Solution. We can use the **Add Data File** button to bring up the file upload window.

<img src="images/data_mapping_1.png" width="500px" />

Use the **Plus Icon** to select the files from your system that you would like to upload. We'll select all of the 10K files that we created earlier.

<img src="images/data_mapping_2.png" width="500px" />

All of the uploaded files will now be visible on the interface. When you click on a file, the first 10 lines of the file will be displayed in the windows on the right. 

<img src="images/data_mapping_3.png" width="500px" />

Above the file preview, we can see come configuration options for the file. 

<img src="images/data_mapping_4.png" width="500px" />

- **File Format** - This specifies the type of file that is being loaded (text or an archive)
- **Delimiter** - The character used to separate columns in the input file
- **End of line** - Character that is used to mark the end of a line
- **Enclosing character** - The character used to surround any fields that may contain a Delimiter
- **Has header** - Checkbox indicates that the first line of the file is a header

We'll check **Has header** for our files because they all have headers. Some of the files also use quotes as an enclosing character, so we will select that as well. The rest of the values can remain their default. Click the **Add** button to add the data file to the Data Mapping.

<img src="images/data_mapping_5.png" width="500px" />

#### Mapping a Vertex

Click the Crossed arrows icon to begin mapping your data file to edges and vertices.

<img src="images/data_mapping_6.png" width="500px" />

Select the Data File, then the Vertex or Edge that you would like to map it to. This will cause the right panel to populate with a table representing our file and one representing the Graph Aspect that we have selected.

We're going to start by mapping the *10K_all_inventors.csv* file to the **Inventor** vertex.

<img src="images/data_mapping_7.png" width="500px" />

Click the field from the file (on the left) that you would like to map, then select the Id or Attribute that you would like to map to from the Vertex table (on the right).

<img src="images/data_mapping_8.png" width="500px" />

#### Token Functions

If you recall from when we created the schema, we don't have a unique ID for our **Inventor**, so we need to create one. For the purpose of this graph, we're going to assume that there are no Inventors who have the same First, Middle, and Last name and we're going to create a unique Id based on the concatenation of those values.

Select the Sigma symbol from the toolbar to bring up the Token function selection window. We're going to use `gsql_concat` with 5 inputs. Why 5 inputs? We want the format of this name to match that of other names in our dataset. In particular the **Examiner Name** from the `all_applicatons.csv` file with is structured: Last,First Middle

Our five values that we need to concatenate are going to be the Last name, a comma character, the First name, a space character, and the Middle name.

<img src="images/data_mapping_9.png" width="500px" /> 
<br>
<img src="images/data_mapping_10.png" width="500px" />

Use the **Add** button to add the Token function to the mapping.

We can map the values just like we did with the values from the .csv file. In order to input a static value like the , or space, we can double click on the name of the input that we'd like to set. The output from the Token function can be mapped to the id of the **Inventor** vertex.

<img src="images/data_mapping_11.png" width="700px" />

#### Mapping an Edge

Mapping a file to an edge is much the same as a **Vertex**. The only distinction here is that you have to specify the **source** and **destination** vertex types. Note that we need to include the **token** function here as well to generate the Id for the **Inventor** vertex.

<img src="images/data_mapping_12.png" width="700px" />

Something important to note is that if a **vertex does not have any attributes**, but **does have an edge** connected to it, then you o**nly need to map to the edge**. This is because an edge can not exits without a source and destination vertex. If you map an edge to both a source and destination vertex, then if either of those vertices does not exist, they will be created. Creation by this process will set the attributes of the source and destination vertices to their default values as we do not have information for them (remember we only have access to the vertex id when mapping an edge). A vertex with no attributes just has an id, and that would be set from the edge creation, so there is no need to map any data directly to that vertex.

For example, the City vertex just has an id, so we only need to map that via the *from_city* edge.

<img src="images/data_mapping_13.png" width="100%" />

<img src="images/data_mapping_14.png" width="100%" />

The **City** vertex only has id available for mapping and that's already mapped by the *from_city* edge mapping.

## Mapping in GSQL

Just like with our Schema, anything that can be set through the GraphStudio interface can be expressed as GSQL code.

As for our data mapping, the end result of what we were doing in GraphStudio is a **Loading Job**. Clicking **Publish Data Mapping** in the interface will cause the TigerGraph server to generate GSQL **Loading Jobs** based off of your data mapping. Doing this through GSQL, we'll just directly write the **Loading Jobs**.

Here's what the **Loading Job** for the `all_inventors` mapping looks like.

```
CREATE LOADING JOB load_all_inventors FOR GRAPH Patents {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Inventor VALUES(gsql_concat($1,",",$3," ",$2), $1, $2, $3) USING SEPARATOR=",", HEADER="false", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE filed_application VALUES($0, gsql_concat($1,",",$3," ",$2), $4) USING SEPARATOR=",", HEADER="false", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE from_city VALUES(gsql_concat($1,",",$3," ",$2), $5) USING SEPARATOR=",", HEADER="false", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE from_region VALUES(gsql_concat($1,",",$3," ",$2), $6) USING SEPARATOR=",", HEADER="false", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE from_country VALUES(gsql_concat($1,",",$3," ",$2), $7) USING SEPARATOR=",", HEADER="false", EOL="\n", QUOTE="double";
    }
```

### Anatomy of a Loading Job

Let's break down that loading job so that it makes a little more sense. 

`CREATE LOADING JOB <LOADING_JOB_NAME> FOR GRAPH <GRAPH_NAME> {`

- `CREATE` - We're adding something to the solution
- `LOADING JOB` - It's an loading job that's being added
- `<LOADING_JOB_NAME>` - The name you want to give to your loading job
- `FOR GRAPH` - The next input will specify which graph the job is for
- `<GRAPH_NAME>` - The name of the graph that the loading job is for

```
DEFINE FILENAME <FILE_VARIABLE_NAME>;
LOAD <FILE_VARIABLE_NAME>
  TO VERTEX|EDGE <VERTEX_TYPE>|<EDGE_TYPE> VALUES(<COLUMN_NUMBER>, ...)
```

- `DEFINE` - Defining a Variable
- `FILENAME` - The type of variable being defined
- `<FILE_VARIABLE_NAME>` - The name of the variable that will represent our input file
- `LOAD` - Specify that the next input is the file that we will be loading
- `<FILE_VARIABLE_NAME>` - The file variable that we are loading
- `TO` - The next input is a vertex type or edge type that the loading job will apply to
- `VERTEX|EDGE` - Specify if the Job is loading to a Vertex or Edge
- `<VERTEX_TYPE>|<EDGE_TYPE>` - The name of the vertex type or edge type being loaded into
- `VALUES` - The next input contains the Column Numbers in order of the fields they represent

**Values Layout**

**Vertex**
`VALUES(PRIMARY_ID, ATTRIBUTE_1, ATTRIBUTE_2, ...)`

**Edge**
`VALUES(SOURCE_ID, DESTINATION_ID, ATTRIBUTE_1, ATTRIBUTE_2, ...)`

In the Values section we're using $0, $1, $2, etc. to denote which column's data should be loaded into which value. **$0** represents the first column in the input data file, **$1** the second column, and so on. Additionally, token functions are represented as such `token_function_name(input1, input2, ...)`.

Additionally we specify any additional options for the loading job after a `USING` statement. For a full list of options check the [Loading Job Documentation](https://docs.tigergraph.com/dev/gsql-ref/ddl-and-loading/creating-a-loading-job#create-loading-job)

### Installing a Loading Job

Again we'll use our pyTigerGraph connection to execute the GSQL that will create our Loading Job.

In [12]:
conn.gsql('''
USE GRAPH Patents
CREATE LOADING JOB load_all_inventors FOR GRAPH Patents {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Inventor VALUES(gsql_concat($1,",",$3," ",$2), $1, $2, $3) USING SEPARATOR=",", HEADER="false", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE filed_application VALUES($0, gsql_concat($1,",",$3," ",$2), $4) USING SEPARATOR=",", HEADER="false", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE from_city VALUES(gsql_concat($1,",",$3," ",$2), $5) USING SEPARATOR=",", HEADER="false", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE from_region VALUES(gsql_concat($1,",",$3," ",$2), $6) USING SEPARATOR=",", HEADER="false", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE from_country VALUES(gsql_concat($1,",",$3," ",$2), $7) USING SEPARATOR=",", HEADER="false", EOL="\n", QUOTE="double";
}
CREATE LOADING JOB load_application_data FOR GRAPH Patents {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Application VALUES($0, $1, $7, $8, $19) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_examiner VALUES($0, $3) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE from_unit VALUES($3, $4) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_class VALUES($0, $5) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_subclass VALUES($0, $6) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE is_subclass VALUES($6, $5) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_status VALUES($0, $9, $10) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_patent VALUES($0, $17, $18) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE is_small VALUES($0, $20) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE follows_ftf VALUES($0, $21) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE at_location VALUES($0, $11, $12) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_PGPUB VALUES($0, $13, $14) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
}
CREATE LOADING JOB load_attorney_agent FOR GRAPH Patents {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Attorney VALUES($5, $0, $2, $1, $3, $4) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_attorney VALUES($7, $5) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_practice_category VALUES($5, $6) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
}
CREATE LOADING JOB load_continuity_children FOR GRAPH Patents {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE has_child VALUES($0, $1, $2) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
}
CREATE LOADING JOB load_continuity_parents FOR GRAPH Patents {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE has_parent VALUES($0, $1, $2) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE is_continuation_type VALUES($0, $3) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
}
CREATE LOADING JOB load_correspondence_address FOR GRAPH Patents {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Correspondence VALUES($9, $1, $9) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO VERTEX Address VALUES(gsql_concat($2,$3,$4), $2, $3, $4) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_address VALUES(gsql_concat($2,$3,$4), $9) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_code VALUES($9 Correspondence, $6 PostalCode) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_city VALUES($9 Correspondence, $5 City) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_region VALUES($9 Correspondence, $7 Region) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_country VALUES($9 Correspondence, $8 Country) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_code VALUES(gsql_concat($2,$3,$4) Address, $6 PostalCode) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_city VALUES(gsql_concat($2,$3,$4) Address, $5 City) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_region VALUES(gsql_concat($2,$3,$4) Address, $7 Region) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_country VALUES(gsql_concat($2,$3,$4) Address, $8 Country) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
}
CREATE LOADING JOB load_event_codes FOR GRAPH Patents {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX EventCode VALUES($0, $1) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
}
CREATE LOADING JOB load_foreign_priority FOR GRAPH Patents {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE has_foreign_parent VALUES($0, $1, $2) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE filed_country VALUES($1, $3) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
}
CREATE LOADING JOB load_pat_term_adj FOR GRAPH Patents {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX PTAEvent VALUES(gsql_uuid_v4($1), $3, $4, $5) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_pta_event VALUES($0, gsql_uuid_v4($1), $2) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE is_extension VALUES(gsql_uuid_v4($1), $7) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
}
CREATE LOADING JOB load_transactions FOR GRAPH Patents {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE has_code VALUES($0, $1, $2) USING SEPARATOR=",", HEADER="false", EOL="\n", QUOTE="double";
}
''')

"Using graph 'Patents'\nSuccessfully created loading jobs: [load_all_inventors].\nSuccessfully created loading jobs: [load_application_data].\nSuccessfully created loading jobs: [load_attorney_agent].\nSuccessfully created loading jobs: [load_continuity_children].\nSuccessfully created loading jobs: [load_continuity_parents].\nSuccessfully created loading jobs: [load_correspondence_address].\nSuccessfully created loading jobs: [load_event_codes].\nSuccessfully created loading jobs: [load_foreign_priority].\nSuccessfully created loading jobs: [load_pat_term_adj].\nSuccessfully created loading jobs: [load_transactions]."

# Loading Data

Now that the loading jobs have been created, we can begin actually loading in data. We'll be stepping away from the GSQL heavy work that we've been using so far and switch back to more python oriented code for loading.

First, we load the data file into a variable.

`uploadFile()` requires 3 inputs:
- `filePath` - The actual data file to load
- `fileTag` - This is the name of the variable that the file will correspond to in the loading job. If you remember, we're using `MyDataSource` as our FILENAME variable in the loading jobs.
- `jobName` - The name of the corresponding loading job to run

In [ ]:
inventors_file = './processed_data/10K_all_inventors.csv'
results = conn.uploadFile(inventors_file, fileTag='MyDataSource', jobName='load_all_inventors')
print(results)

### Programmatic Loading

Loading one file at a time is great and all, but let's let Python handle the heavy lifting. We'll set up a list containing tuples representing the mapping between loading job and file being loaded.

Then we can just iterate through that list and run the corresponding loading jobs.

In [13]:
loading_list = [
    ('load_all_inventors', './processed_data/10K_all_inventors.csv'),
    ('load_application_data', './processed_data/10K_application_data.csv'),
    ('load_attorney_agent', './processed_data/10K_attorney_agent.csv'),
    ('load_continuity_children', './processed_data/10K_continuity_children.csv'),
    ('load_continuity_parents', './processed_data/10K_continuity_parents.csv'),
    ('load_correspondence_address', './processed_data/10K_correspondence_address.csv'),
    ('load_event_codes', './processed_data/10K_event_codes.csv'),
    ('load_foreign_priority', './processed_data/10K_foreign_priority.csv'),
    ('load_pat_term_adj', './processed_data/10K_pat_term_adj.csv'),
    ('load_transactions', './processed_data/10K_transactions.csv')
]

In [ ]:
for job in loading_list:
    print(conn.uploadFile(job[1], fileTag='MyDataSource', jobName=job[0], timeout=600000))

And following that giant wall of text, our data has been loaded. Let's see for sure what was loaded in. (These numbers might take a minute to stabilize)

In [155]:
print("Vertex Counts")
print(conn.getVertexCount("*"))
print("Edge Counts")
print(conn.getEdgeCount("*"))

Vertex Counts
{'ForeignParent': 9897, 'Inventor': 9344, 'FileLocation': 14, 'PGPUBNumber': 352, 'SmallEntity': 5, 'PracticeCategory': 3, 'ExtensionIndicator': 2, 'ApplicationStatus': 45, 'PTASummary': 0, 'PTESummay': 0, 'City': 2931, 'Region': 66, 'Application': 44356, 'Correspondence': 7, 'USPCSubclass': 1615, 'Country': 142, 'ContinuationType': 7, 'PostalCode': 71, 'USPCClass': 403, 'Address': 92, 'EventCode': 2166, 'PTAEvent': 29998, 'First_toFile': 3, 'PatentNumber': 7452, 'ArtUnit': 641, 'WIPONumber': 0, 'Examiner': 2084, 'Attorney': 5667}
Edge Counts
{'has_child': 9924, 'reverse_has_child': 9924, 'has_parent': 9913, 'reverse_has_parent': 9913, 'is_continuation_type': 9138, 'has_class': 10000, 'has_subclass': 10000, 'is_subclass': 6757, 'reverse_is_subclass': 6757, 'has_code': 7641, 'has_pta_event': 9999, 'is_extension': 10000, 'has_start': 0, 'reverse_has_start': 0, 'has_pta_summary': 0, 'has_pte_summary': 0, 'has_status': 9997, 'has_patent': 7452, 'has_examiner': 10000, 'from_un

In [202]:
conn.gsql('''
DROP ALL
''')

'Dropping all, about 1 minute ...\nAbort all active loading jobs\nResetting GPE...\nSuccessfully reset GPE and GSE\nStopping GPE GSE\nSuccessfully stopped GPE GSE in 0.004 seconds\nClearing graph store...\nSuccessfully cleared graph store\nStarting GPE GSE RESTPP\nSuccessfully started GPE GSE RESTPP in 0.096 seconds\nEverything is dropped.'